## Merge datasets

- Clean up and merge the volume and surface area measurements for each dataset
- Save as individual csv



In [10]:
import pandas as pd
import os
import glob
import numpy as np 
#read in MET Data

files_to_process = ['IV/883','IV/934','IV/935','IV/1064','IV/1066','MFI/1067','MFI/1069','MFI/1070','MFI/1381']
path = "./raw_data/"
for data_folder in files_to_process:
    parent_folder = path+data_folder+os.path.sep
    print(f"Processing {parent_folder}")
    files = glob.glob(parent_folder+"*")
    print(f"Files detected {files}")
    filepath = ''
    coord = ''
    surf_area_df = ''
    for file in files:
        print(os.path.splitext(file)[1])
        if os.path.splitext(file)[1] in ['.xlsx','.xls']:
            print(f"xlsx {file}")
            filepath = file 
        elif os.path.splitext(file)[1] in ['.csv']:
            print(f"Coordinates {file}")
            coord =pd.read_csv(file)
        elif os.path.splitext(file)[1] in ['.txt']:
            print(f"Surface area {file}")
            surf_area_df =pd.read_csv(file,sep="\t")
            #strip whitespace
            surf_area_df.columns = surf_area_df.columns.str.strip()

    #fname is parent folder of the excel file
    fname = os.path.dirname(file).split("/")[-1]#file.split("\\")[-2]
    print(f"Processing {fname}")
    # Read CSV with case-insensitive column names
    df = pd.read_excel(filepath, header=0,na_filter='all',engine='openpyxl')
    df.columns = df.columns.astype(str)
    #strip whitespace from column names
    df.columns = df.columns.str.strip()

    #rename column Volume Corrected to Volume corrected 
    df = df.rename(columns={"Volume Corrected": "Volume corrected"})

    #keep columns with following names
    keep_columns = ["MET_ID", "1", "10", "11", "100", "101", "110", "111",
                        "Volume corrected", "min_dist", "max_dist", "mean_dist",
                        "Nearest Vessel Thickness", "Vessel Volume"]
    df = df[keep_columns]
    df = df.dropna(subset=["MET_ID"], how='all')
    df["MET_ID"] = df["MET_ID"].astype(int)


    #1 is Clone1, 10 is Clone2, 11 is Clone3, 100 is Clone4, 101 is Clone5, 110 is Clone6, 111 is Clone7
    #Create additional columns with clone names and binary values if proportion is >0 or a set amount
    proportion_threshold = 0

    #1 if true, 0 if false
    df["Clone1"] = np.where(df["1"] >proportion_threshold, 1,0)
    df["Clone2"] = np.where(df["10"] >proportion_threshold, 1,0)
    df["Clone3"] = np.where(df["11"] >proportion_threshold, 1,0)
    df["Clone4"] = np.where(df["100"] >proportion_threshold, 1,0)
    df["Clone5"] = np.where(df["101"] >proportion_threshold, 1,0)
    df["Clone6"] = np.where(df["110"] >proportion_threshold, 1,0)
    df["Clone7"] = np.where(df["111"] >proportion_threshold, 1,0)

    #merge MET data with coordinates and merge based on MET_ID and Label indices
    idx_data = "MET_ID"
    idx_coord = "Label"
    merged_df = df.merge(coord,left_on='MET_ID', right_on='Label',how='left')
    merged_df = merged_df.merge(surf_area_df,left_on='MET_ID', right_on='MetNum_that_touches',how='left')
      
    #Calculate centroid of bounding box, bounding box column names are Box.X.Min, Box.X.Max and so on for Y and Z
    merged_df["centroid_x"] = (merged_df["Box.X.Min"] + merged_df["Box.X.Max"])/2
    merged_df["centroid_y"] = (merged_df["Box.Y.Min"] + merged_df["Box.Y.Max"])/2
    merged_df["centroid_z"] = (merged_df["Box.Z.Min"] + merged_df["Box.Z.Max"])/2
    #remove the columns with name Label; keep bounding box
    merged_df = merged_df.drop(columns=["Label"]) #
    merged_df.to_csv(f"./merged/{fname}_merged_surf_area.csv",index=False,sep="\t")
    print(f"Processed {fname}")
    parent_folder=''


Processing ./raw_data/IV/883\
Files detected ['./raw_data/IV/883\\20240115_883_FINAL_Definitely_final_version2_end.xlsx', './raw_data/IV/883\\tmp_metBbox.csv', './raw_data/IV/883\\VesselSurfaceAreaMeasurements_883.txt']
.xlsx
xlsx ./raw_data/IV/883\20240115_883_FINAL_Definitely_final_version2_end.xlsx
.csv
Coordinates ./raw_data/IV/883\tmp_metBbox.csv
.txt
Surface area ./raw_data/IV/883\VesselSurfaceAreaMeasurements_883.txt
Processing 883
Processed 883
Processing ./raw_data/IV/934\
Files detected ['./raw_data/IV/934\\20240116_934_FINAL_Definitely_final_version2_end.xlsx', './raw_data/IV/934\\tmp_metBbox.csv', './raw_data/IV/934\\VesselSurfaceAreaMeasurements_934.txt']
.xlsx
xlsx ./raw_data/IV/934\20240116_934_FINAL_Definitely_final_version2_end.xlsx
.csv
Coordinates ./raw_data/IV/934\tmp_metBbox.csv
.txt
Surface area ./raw_data/IV/934\VesselSurfaceAreaMeasurements_934.txt
Processing 934
Processed 934
Processing ./raw_data/IV/935\
Files detected ['./raw_data/IV/935\\20240115_935_FINAL_D